In [1]:
!pip install polaris-ml

     |████████████████████████████████| 133 kB 13.1 MB/s 
     |████████████████████████████████| 307 kB 46.2 MB/s 
     |████████████████████████████████| 53 kB 2.1 MB/s 
     |████████████████████████████████| 15.5 MB 41.3 MB/s 
     |████████████████████████████████| 55 kB 3.7 MB/s 
     |████████████████████████████████| 2.8 MB 44.9 MB/s 
     |████████████████████████████████| 524 kB 44.6 MB/s 
     |████████████████████████████████| 3.1 MB 44.5 MB/s 
     |████████████████████████████████| 143 kB 53.1 MB/s 
     |████████████████████████████████| 42 kB 823 kB/s 
     |████████████████████████████████| 503 kB 47.3 MB/s 
     |████████████████████████████████| 199 kB 54.5 MB/s 
     |████████████████████████████████| 258 kB 50.9 MB/s 
     |████████████████████████████████| 4.5 MB 44.5 MB/s 
     |████████████████████████████████| 40 kB 6.1 MB/s 
     |████████████████████████████████| 832 kB 40.0 MB/s 
     |████████████████████████████████| 54 kB 2.2 MB/s 
     |█████████████████

In [1]:
import os
import pandas as pd
import numpy as np
os.listdir("./")
lista_archivos_lightsail = [x for x in os.listdir("./") if x[-3:] == "txt"]
lista_archivos_marsexpress = ["./mars_express/" + x for x in os.listdir("./mars_express/") if x[-3:] == "csv"]
print(lista_archivos_lightsail[:5])
print(lista_archivos_marsexpress[:5])
#os.getcwd()
#os.path.getsize("2127458.txt")
#os.path.isfile("2127458.txt")



[]
['./mars_express/context--2008-08-22_2010-07-10--ftl.csv', './mars_express/context--2008-08-22_2010-07-10--evtf.csv', './mars_express/context--2012-05-27_2014-04-14--evtf.csv', './mars_express/context--2008-08-22_2010-07-10--saaf.csv', './mars_express/context--2010-07-10_2012-05-27--saaf.csv']


In [3]:
#DATASET PARA LECTURA HUMANA DEL LIGHTSAIL

def read_human_from_txt_lightsail(lista_archivos):

  etiquetas=["Observed_at", "Posted_at"]

  etiquetas2=[]

  dataset={}

  dataset[etiquetas[0]]=[]
  dataset[etiquetas[1]]=[]

  valores = []
  valores2 = []

  archivo1 = open(lista_archivos[0], "r")

  for linea in archivo1:
    if "Observed at" in linea:
      valor = linea.split("at:")[1].strip()
      dataset["Observed_at"].append(valor)
    elif "Posted at" in linea:
      valor = linea.split("at:")[1].strip()
      dataset["Posted_at"].append(valor)
    elif ":" in linea and "=" in linea:
      etiqueta = linea.split(":")[1].split("=")[0].strip().replace(" ", "_").replace("[", "_").replace("]", "_").replace("<", "_")
      valor = linea.split(":")[1].split("=")[1].split("[")[0].strip()
      dataset[etiqueta]=[]
      dataset[etiqueta].append(valor)

  for nombre_archivo in lista_archivos[1:]:    
    archivo = open(nombre_archivo, "r")
    for linea in archivo: 
      if "Observed at" in linea:
        valor = linea.split("at:")[1].strip()
        dataset["Observed_at"].append(valor)
      elif "Posted at" in linea:
        valor = linea.split("at:")[1].strip()
        dataset["Posted_at"].append(valor)
      elif ":" in linea and "=" in linea:
        etiqueta = linea.split(":")[1].split("=")[0].strip().replace(" ", "_").replace("[", "_").replace("]", "_").replace("<", "_")
        valor = linea.split(":")[1].split("=")[1].split("[")[0].strip()
        dataset[etiqueta].append(valor)

  df = pd.DataFrame(dataset.values(), dataset.keys())
  df = df.transpose()
  return df




In [4]:
#LECTURA DE LOS DATASETS DE LIGHTSAIL Y MARS EXPRESS
from datetime import datetime

def transform_unix_to_utc(series):
  if len(series) > 0:
    new_series = [str(datetime.utcfromtimestamp(x/1000)) for x in series]
    return new_series
  else:
    return []

def read_from_txt_lightsail():
  etiquetas=[]
  dataset={}
  archivo1 = open(lista_archivos[0], "r")
  kvp_form = False

  for linea in archivo1:
    if "KVP form:" in linea:
      kvp_form = True
    if kvp_form == True and "=" in linea:
        etiqueta = linea.split("=")[0].strip()
        valor = float(linea.split("=")[1].split(",")[0].strip())
        dataset[etiqueta]=[]
        dataset[etiqueta].append(valor)

  for nombre_archivo in lista_archivos[1:]:    
    archivo = open(nombre_archivo, "r")
    kvp_form = False
    for linea in archivo: 
      if "KVP form:" in linea:
        kvp_form = True
      if kvp_form == True and "=" in linea:
        etiqueta = linea.split("=")[0].strip()
        valor = float(linea.split("=")[1].split(",")[0].strip())
        dataset[etiqueta].append(valor)

  df = pd.DataFrame(dataset.values(), dataset.keys())
  df = df.transpose()
  return df


def read_from_ut_ms(lista_archivos):
  columnas = []
  for archivo in lista_archivos:
    df1 = pd.read_csv(archivo)
    columnas += list(df1.columns)
    columnas = list(set(columnas))
  
  df = pd.DataFrame(columns=columnas)
  df = df.loc[:,~df.columns.duplicated()]
  for archivo in lista_archivos:
    df2 = pd.read_csv(archivo)
    df2["date"] = transform_unix_to_utc(df2["ut_ms"])
    df = df.append(df2)    
  return df

def read_from_utb_ms(lista_archivos):
  columnas = []
  for archivo in lista_archivos:
    df1 = pd.read_csv(archivo)
    columnas = list(df1.columns)
    columnas.append("date")
  df = pd.DataFrame(columns=columnas)
  for archivo in lista_archivos:
    df2 = pd.read_csv(archivo)
    df2["date"] = transform_unix_to_utc(df2["utb_ms"])
    df = df.append(df2)
  return df

def read_from_csv_mars_express(lista_archivos):

  lista_archivos_saaf = [x for x in lista_archivos_marsexpress if x.split("--")[-1].split(".")[0] == "saaf"]
  lista_archivos_dmop = [x for x in lista_archivos_marsexpress if x.split("--")[-1].split(".")[0] == "dmop"]
  lista_archivos_ftl = [x for x in lista_archivos_marsexpress if x.split("--")[-1].split(".")[0] == "ftl"]
  lista_archivos_evtf = [x for x in lista_archivos_marsexpress if x.split("--")[-1].split(".")[0] == "evtf"]
  lista_archivos_ltdata = [x for x in lista_archivos_marsexpress if x.split("--")[-1].split(".")[0] == "ltdata"]

  df_saaf = read_from_ut_ms(lista_archivos_saaf)
  df_dmop = read_from_ut_ms(lista_archivos_dmop)
  df_ftl = read_from_utb_ms(lista_archivos_ftl)
  df_evtf = read_from_ut_ms(lista_archivos_evtf)
  df_ltdata = read_from_ut_ms(lista_archivos_ltdata)

  all_columns = list(df_saaf.columns) + list(df_dmop.columns) + list(df_ftl.columns) + list(df_evtf.columns) + list(df_ltdata.columns)
  all_columns = list(set(all_columns))

  df = pd.DataFrame(columns=all_columns)
  df = df.append(df_saaf, ignore_index=True)
  df = df.append(df_dmop, ignore_index=True)
  df = df.append(df_ftl, ignore_index=True)
  df = df.append(df_evtf, ignore_index=True)
  df = df.append(df_ltdata, ignore_index=True).sort_values("date")


  return df




In [5]:
#COMPROBAR RESULTADOS ANTES DE UTILIZAR POLARIS
df = read_machine_from_csv(lista_archivos_marsexpress)
display(df)

,sy,description,solarconstantmars,sz,utb_ms,type,occultationduration_min,sx,date,sa,ute_ms,subsystem,sunmarsearthangle_deg,eclipseduration_min,earthmars_km,flagcomms,sunmars_km,ut_ms
3155780,NaN,NaN,522.264,NaN,NaN,NaN,27.4,NaN,2008-08-22 00:00:00,NaN,NaN,NaN,19.5651,4.16667,3.55756e+08,NaN,2.41939e+08,1219363200000
2672561,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008-08-22 00:00:11,NaN,NaN,AXXX301A,NaN,NaN,NaN,NaN,NaN,1219363211000
2894676,NaN,NaN,NaN,NaN,1219363213000,EARTH,NaN,NaN,2008-08-22 00:00:13,NaN,1219365494000,NaN,NaN,NaN,NaN,False,NaN,NaN
0,90.32,NaN,NaN,104.55,NaN,NaN,NaN,14.55,2008-08-22 00:00:13,0.32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1219363213000
1,90.34,NaN,NaN,104.55,NaN,NaN,NaN,14.56,2008-08-22 00:00:35,0.34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1219363235000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3002420,NaN,MSL_/_RANGE_06000KM_START,NaN,NaN,NaN,NaN,NaN,NaN,2014-04-13 23:56:47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1397433407000
2364778,88.59,NaN,NaN,89.32,NaN,NaN,NaN,1.57,2014-04-13 23:57:44,1.41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1397433464000
3002421,NaN,4000_KM_DESCEND,NaN,NaN,NaN,NaN,NaN,NaN,2014-04-13 23:58:38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1397433518000
2364779,88.59,NaN,NaN,89.32,NaN,NaN,NaN,1.57,2014-04-13 23:58:44,1.41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1397433524000


In [120]:
#GUARDAR CSV DEL LIGHTSAIL2 PARA PROCESAMIENTO DE POLARIS
df_csv = df.to_csv('lightsail_dataset.csv',sep=',', index=False)


In [6]:
#GUARDAR CSV DEL MARS EXPRESS PARA PROCESAMIENTO DE POLARIS
df_csv = df.to_csv('marsexpress_dataset.csv',sep=',', index=False)

In [7]:
#ANALISYS.PY DE POLARIS, PARA PODER PERSONALIZARLO SEGÚN EL DATASET CONVENGA

"""
Module to launch different data analysis.
"""
import logging

from fets.math import TSIntegrale
from mlflow import set_experiment

from polaris.data.graph import PolarisGraph
from polaris.data.readers import read_polaris_data
from polaris.dataset.metadata import PolarisMetadata
from polaris.learn.feature.extraction import create_list_of_transformers, \
    extract_best_features
from polaris.learn.predictor.cross_correlation import XCorr
from polaris.learn.predictor.cross_correlation_configurator import \
    CrossCorrelationConfigurator

LOGGER = logging.getLogger(__name__)


class NoFramesInInputFile(Exception):
    """Raised when frames dataframe is empty"""


def feature_extraction(input_file, param_col):
    """
    Start feature extraction using the given settings.

        :param input_file: Path of a CSV file that will be
            converted to a dataframe
        :type input_file: str
        :param param_col: Target column name
        :type param_col: str
    """
    # Create a small list of two transformers which will generate two
    # different pipelines
    transformers = create_list_of_transformers(["5min", "15min"], TSIntegrale)

    # Extract the best features of the two pipelines
    out = extract_best_features(input_file,
                                transformers,
                                target_column=param_col,
                                time_unit="ms")

    # out[0] is the FeatureImportanceOptimization object
    # from polaris.learn.feature.selection
    # pylint: disable=E1101
    print(out[0].best_features)


# pylint: disable-msg=too-many-arguments
def cross_correlate(input_file,
                    output_graph_file=None,
                    xcorr_configuration_file=None,
                    graph_link_threshold=0.1,
                    use_gridsearch=False,
                    csv_sep=',',
                    force_cpu=False):
    """
    Catch linear and non-linear correlations between all columns of the
    input data.

        :param input_file: CSV or JSON file path that will be
            converted to a dataframe
        :type input_file: str
        :param output_graph_file: Output file path for the generated graph.
            It will overwrite if the file already exists. Defaults to None,
            which is'/tmp/polaris_graph.json'
        :type output_graph_file: str, optional
        :param xcorr_configuration_file: XCorr configuration file path,
            defaults to None. Refer to CrossCorrelationConfigurator for
            the default parameters
        :type xcorr_configuration_file: str, optional
        :param graph_link_threshold: Minimum link value to be considered
            as a link between two nodes
        :type graph_link_threshold: float, optional
        :param use_gridsearch: Use grid search for the cross correlation.
            If this is set to False, then it will just use regression.
            Defaults to False
        :type use_gridsearch: bool, optional
        :param csv_sep: The character that separates the columns inside of
            the CSV file, defaults to ','
        :type csv_sep: str, optional
        :param force_cpu: Force CPU for cross corelation, defaults to False
        :type force_cpu: bool, optional
        :raises NoFramesInInputFile: If there are no frames in the converted
            dataframe
    """
    # Reading input file - index is considered on first column
    metadata, dataframe = read_polaris_data(input_file, csv_sep)

    if dataframe.empty:
        LOGGER.error("Empty list of frames -- nothing to learn from!")
        raise NoFramesInInputFile

    input_data = normalize_dataframe(dataframe)
    source = metadata['satellite_name']

    set_experiment(experiment_name=source)

    xcorr_configurator = CrossCorrelationConfigurator(
        xcorr_configuration_file=xcorr_configuration_file,
        use_gridsearch=use_gridsearch,
        force_cpu=force_cpu)

    # Creating and fitting cross-correlator
    xcorr = XCorr(metadata, xcorr_configurator.get_configuration())
    xcorr.fit(input_data)

    if output_graph_file is None:
        output_graph_file = "/tmp/polaris_graph.json"

    metadata = PolarisMetadata({"satellite_name": source})
    graph = PolarisGraph(metadata=metadata)
    graph.from_heatmap(xcorr.importances_map, graph_link_threshold)
    with open(output_graph_file, 'w') as graph_file:
        graph_file.write(graph.to_json())


def normalize_dataframe(dataframe):
    """
        Apply dataframe modification so it's compatible
        with the learn module. The time column is first
        set as the index of the dataframe. Then, we drop
        the time column.

        :param dataframe: The pandas dataframe to normalize
        :type dataframe: pd.DataFrame
        :return: Pandas dataframe normalized
        :rtype: pd.DataFrame
    """
    dataframe.index = dataframe.date
    dataframe.drop(['date'], axis=1, inplace=True)

    return dataframe


In [ ]:
#GENERA EL GRAFO DE LIGHTSAIL2
cross_correlate("lightsail_dataset.csv")


In [8]:
#GENERA EL GRAFO DE MARS EXPRESS
cross_correlate("marsexpress_dataset.csv")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.
2022/01/10 21:18:19 INFO mlflow.tracking.fluent: Experiment with name 'marsexpress_dataset' does not exist. Creating a new experiment.
